This is attempt 2 at using the keras bert model to train the dataset and predict sentiment labels fro phrases in the test dataset. This time I want to increase the accuracy.

The training and testing data was cut into only 20 random rows because the kernal would have died if all the rows were processed. 

In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load the training dataset
train_data = pd.read_csv('train.tsv', delimiter='\t')  

# Load the testing dataset
test_data = pd.read_csv('test.tsv', delimiter='\t')  

# random sample of 20 rows for train/test data
random_sample_train_data = train_data.sample(n=500, random_state=42)
random_sample_test_data = test_data.sample(n=200, random_state=42)

# Initialize a BERT tokenizer from the pre-trained model 'bert-base-uncased' and set the maximum length of the tokenized sequence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

# Preprocessing training data
# Tokenize the 'Phrase' column of the training data using the BERT tokenizer
X_train_tokenized = tokenizer(
    random_sample_train_data['Phrase'].tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='tf'
)

# Convert the tokenized training data into a hashable format (dictionary of TensorFlow tensors), so we can use .fit() later on
X_train_hashable = {
    'input_ids': X_train_tokenized['input_ids'],
    'token_type_ids': X_train_tokenized['token_type_ids'],
    'attention_mask': X_train_tokenized['attention_mask'],
}

y_train = random_sample_train_data['Sentiment'].values


# Preprocess testing data
# Tokenize the 'Phrase' column of the testing data using the same BERT tokenizer
X_test_tokenized = tokenizer(
    random_sample_test_data['Phrase'].tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='tf'
)

# Convert the tokenized testing data into a hashable format (dictionary of TensorFlow tensors)
X_test_hashable = {
    'input_ids': X_test_tokenized['input_ids'],
    'token_type_ids': X_test_tokenized['token_type_ids'],
    'attention_mask': X_test_tokenized['attention_mask'],
}


# Load the pre-trained BERT model for sequence classification, set num_labels to 5
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Compile the model
bert_model.compile(optimizer=Adam(learning_rate=2e-5), 
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[SparseCategoricalAccuracy()])

# Train the model
bert_model.fit(X_train_hashable, y_train, epochs=3, batch_size=32)


# Predict the sentiment labels of the testing data
predictions = bert_model.predict(X_test_hashable) # will return unnormalized probability predictions (aka decimal values)
logits = predictions.logits
probabilities = tf.nn.softmax(logits) # use softmax() to convert raw scores (logits) into a probability distribution (class labels 0 1 2 3 4)
predicted_class = tf.argmax(probabilities, axis=-1)

print("Predicted labels for testing data: ", np.array(predicted_class).tolist())



2023-12-11 14:46:16.250822: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:46:16.555641: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 14:46:16.555697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 14:46:16.624509: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 14:46:16.767613: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:46:16.769901: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/3
 1/16 [>.............................] - ETA: 5:55 - loss: 1.6343 - sparse_categorical_accuracy: 0.1875

The results automatically look more accurate as it is not just 2's this time but a mix of numbers. However, without turning in my results, we wont really know how accurate this model is. That is why I used the model to predict the training dataset and then compare the predictions to the actual sentiment is has. 

In [ ]:
result_df = pd.DataFrame({
    'PhraseID': random_sample_test_data['PhraseId'],
    'Sentiment': predicted_labels
})


In [5]:
test_data

PhraseId  SentenceId                                             Phrase
0        156061        8545  An intermittently pleasing but mostly routine ...
1        156062        8545  An intermittently pleasing but mostly routine ...
2        156063        8545                                                 An
3        156064        8545  intermittently pleasing but mostly routine effort
4        156065        8545         intermittently pleasing but mostly routine
...         ...         ...                                                ...
66287    222348       11855             A long-winded , predictable scenario .
66288    222349       11855               A long-winded , predictable scenario
66289    222350       11855                                    A long-winded ,
66290    222351       11855                                      A long-winded
66291    222352       11855                               predictable scenario

[66292 rows x 3 columns]

In [6]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow as tf

# Load the training dataset
train_data = pd.read_csv('train.tsv', delimiter='\t')  

# Load the testing dataset
test_data = pd.read_csv('test.tsv', delimiter='\t')  

# Initialize a BERT tokenizer from the pre-trained model 'bert-base-uncased' and set the maximum length of the tokenized sequence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

# Preprocess training data
# Tokenize the 'Phrase' column of the training data using the BERT tokenizer
X_train_tokenized = tokenizer(
    train_data['Phrase'].tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='tf'
)

# Convert the tokenized training data into a hashable format (dictionary of TensorFlow tensors)
X_train_hashable = {
    'input_ids': X_train_tokenized['input_ids'],
    'token_type_ids': X_train_tokenized['token_type_ids'],
    'attention_mask': X_train_tokenized['attention_mask'],
}

y_train = train_data['Sentiment'].values

# Preprocess testing data
# Tokenize the 'Phrase' column of the testing data using the same BERT tokenizer
X_test_tokenized = tokenizer(
    test_data['Phrase'].tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='tf'
)

# Convert the tokenized testing data into a hashable format (dictionary of TensorFlow tensors)
X_test_hashable = {
    'input_ids': X_test_tokenized['input_ids'],
    'token_type_ids': X_test_tokenized['token_type_ids'],
    'attention_mask': X_test_tokenized['attention_mask'],
}

# Load the pre-trained BERT model for sequence classification, set num_labels to 5
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Compile the model
bert_model.compile(optimizer=Adam(learning_rate=2e-5), 
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[SparseCategoricalAccuracy()])

# Train the model on all training data
bert_model.fit(X_train_hashable, y_train, epochs=3, batch_size=32)

# Predict the sentiment labels for all rows of the testing data
predictions = bert_model.predict(X_test_hashable)
predicted_class = np.argmax(predictions.logits, axis=-1)

print("Predicted labels for testing data:", predicted_class)


ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow as tf

# Load the training dataset
train_data = pd.read_csv('train.tsv', delimiter='\t')  

# Load the testing dataset
test_data = pd.read_csv('test.tsv', delimiter='\t')  

# Initialize a BERT tokenizer from the pre-trained model 'bert-base-uncased' and set the maximum length of the tokenized sequence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

# Preprocess training data
# Tokenize the 'Phrase' column of the training data using the BERT tokenizer
X_train_tokenized = tokenizer(
    train_data['Phrase'].astype(str).tolist(),  # Ensure the 'Phrase' column is converted to strings
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='tf'
)

# Convert the tokenized training data into a hashable format (dictionary of TensorFlow tensors)
X_train_hashable = {
    'input_ids': X_train_tokenized['input_ids'],
    'token_type_ids': X_train_tokenized['token_type_ids'],
    'attention_mask': X_train_tokenized['attention_mask'],
}

y_train = train_data['Sentiment'].values

# Preprocess testing data
# Tokenize the 'Phrase' column of the testing data using the same BERT tokenizer
X_test_tokenized = tokenizer(
    test_data['Phrase'].astype(str).tolist(),  # Ensure the 'Phrase' column is converted to strings
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='tf'
)

# Convert the tokenized testing data into a hashable format (dictionary of TensorFlow tensors)
X_test_hashable = {
    'input_ids': X_test_tokenized['input_ids'],
    'token_type_ids': X_test_tokenized['token_type_ids'],
    'attention_mask': X_test_tokenized['attention_mask'],
}

# Load the pre-trained BERT model for sequence classification, set num_labels to 5
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Compile the model
bert_model.compile(optimizer=Adam(learning_rate=2e-5), 
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[SparseCategoricalAccuracy()])

# Train the model on all training data
bert_model.fit(X_train_hashable, y_train, epochs=3, batch_size=32)

# Predict the sentiment labels for all rows of the testing data
predictions = bert_model.predict(X_test_hashable)
predicted_class = np.argmax(predictions.logits, axis=-1)

print("Predicted labels for testing data:", predicted_class)


2023-12-11 14:41:07.482557: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:41:07.784695: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 14:41:07.784732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 14:41:07.850753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 14:41:07.989502: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:41:07.992019: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/3


In [1]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow as tf

# Load the training dataset
train_data = pd.read_csv('train.tsv', delimiter='\t')  

# Load the testing dataset
test_data = pd.read_csv('test.tsv', delimiter='\t')  

# Initialize a BERT tokenizer from the pre-trained model 'bert-base-uncased' and set the maximum length of the tokenized sequence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

# Function to process data in batches
def process_data_in_batches(data, batch_size=1000):
    num_batches = len(data) // batch_size + 1
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size: (i + 1) * batch_size]
        yield batch_data

# Preprocess training data
# Tokenize the 'Phrase' column of the training data using the BERT tokenizer
X_train_hashable_list = []
for batch_data in process_data_in_batches(train_data):
    X_train_tokenized = tokenizer(
        batch_data['Phrase'].astype(str).tolist(),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf'
    )
    X_train_hashable_list.append({
        'input_ids': X_train_tokenized['input_ids'],
        'token_type_ids': X_train_tokenized['token_type_ids'],
        'attention_mask': X_train_tokenized['attention_mask'],
    })

X_train_hashable = {
    'input_ids': tf.concat([batch['input_ids'] for batch in X_train_hashable_list], axis=0),
    'token_type_ids': tf.concat([batch['token_type_ids'] for batch in X_train_hashable_list], axis=0),
    'attention_mask': tf.concat([batch['attention_mask'] for batch in X_train_hashable_list], axis=0),
}

y_train = train_data['Sentiment'].values

# Preprocess testing data
# Tokenize the 'Phrase' column of the testing data using the same BERT tokenizer
X_test_hashable_list = []
for batch_data in process_data_in_batches(test_data):
    X_test_tokenized = tokenizer(
        batch_data['Phrase'].astype(str).tolist(),
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors='tf'
    )
    X_test_hashable_list.append({
        'input_ids': X_test_tokenized['input_ids'],
        'token_type_ids': X_test_tokenized['token_type_ids'],
        'attention_mask': X_test_tokenized['attention_mask'],
    })

X_test_hashable = {
    'input_ids': tf.concat([batch['input_ids'] for batch in X_test_hashable_list], axis=0),
    'token_type_ids': tf.concat([batch['token_type_ids'] for batch in X_test_hashable_list], axis=0),
    'attention_mask': tf.concat([batch['attention_mask'] for batch in X_test_hashable_list], axis=0),
}

# Load the pre-trained BERT model for sequence classification, set num_labels to 5
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Compile the model
bert_model.compile(optimizer=Adam(learning_rate=2e-5), 
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[SparseCategoricalAccuracy()])

# Train the model on all training data
bert_model.fit(X_train_hashable, y_train, epochs=3, batch_size=32)

# Predict the sentiment labels for all rows of the testing data
predictions = bert_model.predict(X_test_hashable)
predicted_class = np.argmax(predictions.logits, axis=-1)

print("Predicted labels for testing data:", predicted_class)


2023-12-11 14:21:12.735073: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:21:13.076131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 14:21:13.076173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 14:21:13.144194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 14:21:13.290997: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:21:13.293923: I tensorflow/core/platform/cpu_feature_guard.cc:1

InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_157_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Dimension 1 in both shapes must be equal: shape[0] = [1000,48] vs. shape[1] = [1000,44] [Op:ConcatV2] name: concat

In [2]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow as tf

# Load the training dataset
train_data = pd.read_csv('train.tsv', delimiter='\t')  

# Load the testing dataset
test_data = pd.read_csv('test.tsv', delimiter='\t')  

# Initialize a BERT tokenizer from the pre-trained model 'bert-base-uncased' and set the maximum length of the tokenized sequence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

# Function to process data in batches
def process_data_in_batches(data, tokenizer, max_length, batch_size=1000):
    num_batches = len(data) // batch_size + 1
    hashable_list = []
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size: (i + 1) * batch_size]
        X_tokenized = tokenizer(
            batch_data['Phrase'].astype(str).tolist(),
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='tf'
        )
        hashable_list.append({
            'input_ids': X_tokenized['input_ids'],
            'token_type_ids': X_tokenized['token_type_ids'],
            'attention_mask': X_tokenized['attention_mask'],
        })
    return hashable_list

# Preprocess training data
X_train_hashable_list = process_data_in_batches(train_data, tokenizer, max_length)

# Concatenate the batches
X_train_hashable = {
    'input_ids': tf.concat([batch['input_ids'] for batch in X_train_hashable_list], axis=0),
    'token_type_ids': tf.concat([batch['token_type_ids'] for batch in X_train_hashable_list], axis=0),
    'attention_mask': tf.concat([batch['attention_mask'] for batch in X_train_hashable_list], axis=0),
}

y_train = train_data['Sentiment'].values

# Preprocess testing data
X_test_hashable_list = process_data_in_batches(test_data, tokenizer, max_length)

# Concatenate the batches
X_test_hashable = {
    'input_ids': tf.concat([batch['input_ids'] for batch in X_test_hashable_list], axis=0),
    'token_type_ids': tf.concat([batch['token_type_ids'] for batch in X_test_hashable_list], axis=0),
    'attention_mask': tf.concat([batch['attention_mask'] for batch in X_test_hashable_list], axis=0),
}

# Load the pre-trained BERT model for sequence classification, set num_labels to 5
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Compile the model
bert_model.compile(optimizer=Adam(learning_rate=2e-5), 
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[SparseCategoricalAccuracy()])

# Train the model on all training data
bert_model.fit(X_train_hashable, y_train, epochs=3, batch_size=32)

# Predict the sentiment labels for all rows of the testing data
predictions = bert_model.predict(X_test_hashable)
predicted_class = np.argmax(predictions.logits, axis=-1)

print("Predicted labels for testing data:", predicted_class)


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_157_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Dimension 1 in both shapes must be equal: shape[0] = [1000,48] vs. shape[1] = [1000,44] [Op:ConcatV2] name: concat

In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import tensorflow as tf

# Load the training dataset
train_data = pd.read_csv('train.tsv', delimiter='\t')  

# Load the testing dataset
test_data = pd.read_csv('test.tsv', delimiter='\t')  

# Initialize a BERT tokenizer from the pre-trained model 'bert-base-uncased' and set the maximum length of the tokenized sequence
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to process data in batches
def process_data_in_batches(data, tokenizer, batch_size=1000):
    num_batches = len(data) // batch_size + 1
    hashable_list = []
    max_length = 0
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size: (i + 1) * batch_size]
        X_tokenized = tokenizer(
            batch_data['Phrase'].astype(str).tolist(),
            truncation=True,
            padding=True,
            return_tensors='tf'
        )
        hashable_list.append({
            'input_ids': X_tokenized['input_ids'],
            'token_type_ids': X_tokenized['token_type_ids'],
            'attention_mask': X_tokenized['attention_mask'],
        })
        max_length = max(max_length, X_tokenized['input_ids'].shape[1])

    # Pad sequences to the maximum length
    for i in range(num_batches):
        batch_size_i = hashable_list[i]['input_ids'].shape[0]
        pad_length = max_length - hashable_list[i]['input_ids'].shape[1]
        for key in hashable_list[i]:
            hashable_list[i][key] = tf.pad(hashable_list[i][key], paddings=[[0, 0], [0, pad_length]])

    return hashable_list

# Preprocess training data
X_train_hashable_list = process_data_in_batches(train_data, tokenizer)

# Concatenate the batches
X_train_hashable = {
    'input_ids': tf.concat([batch['input_ids'] for batch in X_train_hashable_list], axis=0),
    'token_type_ids': tf.concat([batch['token_type_ids'] for batch in X_train_hashable_list], axis=0),
    'attention_mask': tf.concat([batch['attention_mask'] for batch in X_train_hashable_list], axis=0),
}

y_train = train_data['Sentiment'].values

# Preprocess testing data
X_test_hashable_list = process_data_in_batches(test_data, tokenizer)

# Concatenate the batches
X_test_hashable = {
    'input_ids': tf.concat([batch['input_ids'] for batch in X_test_hashable_list], axis=0),
    'token_type_ids': tf.concat([batch['token_type_ids'] for batch in X_test_hashable_list], axis=0),
    'attention_mask': tf.concat([batch['attention_mask'] for batch in X_test_hashable_list], axis=0),
}

# Load the pre-trained BERT model for sequence classification, set num_labels to 5
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

# Compile the model
bert_model.compile(optimizer=Adam(learning_rate=2e-5), 
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics=[SparseCategoricalAccuracy()])

# Train the model on all training data
bert_model.fit(X_train_hashable, y_train, epochs=3, batch_size=32)

# Predict the sentiment labels for all rows of the testing data
predictions = bert_model.predict(X_test_hashable)
predicted_class = np.argmax(predictions.logits, axis=-1)

print("Predicted labels for testing data:", predicted_class)


2023-12-11 14:37:52.210625: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:37:52.512808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 14:37:52.512880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 14:37:52.581270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 14:37:52.724617: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 14:37:52.726405: I tensorflow/core/platform/cpu_feature_guard.cc:1

Epoch 1/3
